# ESD for Pytorch pretrained models
http://pytorch.org/docs/master/torchvision/models.html

We don't normalize by 1/N for these models ?  IDK why

In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn

import numpy as np
import powerlaw

import sklearn
from sklearn.decomposition import TruncatedSVD

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

In [2]:
import import_ipynb
import RMT_Util

importing notebook from RMT_Util.ipynb


/Users/charlesmartin14/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/charlesmartin14/.local/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


3.6.5 |Anaconda custom (64-bit)| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
numpy version 1.14.2
scipy version 1.0.1
tensforflow version 1.5.0-dev20171116
keras version 2.1.5
sklearn version 0.19.1
powerlaw version 1.4.3


In [3]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [5,5]

### Universality Classes:  Is this right ?

#### Truncated Power Law
$$\mu>2;\;\;\rho(\lambda)\sim\lambda^{-1-\mu}$$

$$\alpha=1+\mu,\;\;\mu=\alpha-1,\;\;\alpha>3$$

#### Power Law


$$\mu<2;\;\;\rho(\lambda)\sim\lambda^{-1-\mu/2}$$

$$\alpha=1+\dfrac{\mu}{2};\;\;\mu=2(\alpha-1)$$


In [4]:
def analyze_model(model, plot=True):
    pretrained_model = model(pretrained=True)
    
    model_name = model.__name__
    model_name = model_name.replace("_",' ')
    
    for im, m in enumerate(pretrained_model.modules()):
        if isinstance(m, nn.Linear):
                
            W = np.array(m.weight.data.clone().cpu())
            M, N = np.min(W.shape), np.max(W.shape)
            Q=N/M 
            
            sv, _ = RMT_Util.singular_spectrum(W)
            evals =(1/N)*sv*sv
            
            # RMT_Util.fit_power_law
            fit = powerlaw.Fit(evals, xmax=np.max(evals), verbose=False)  
            alpha = fit.alpha
            D = fit.D
            
            dist = RMT_Util.best_dist(fit)

            if plot:
                RMT_Util.plot_ESD_and_fit(model=None,eigenvalues=evals, weightfile=None, layer=None, 
                                Q=Q, num_spikes=0, sigma=None, color='blue')
                plt.show()
            
            line = "{} & {}& {:.3g} & $({}\\times {})$ & {:.3g} & {:.3g} & {} \\\\ ".format(model_name,im, Q, np.min(W.shape),np.max(W.shape),alpha,D,dist)
            print(line)
            model_name = " "
    print("\\hline")


In [5]:
for model in [models.alexnet]:
    analyze_model(model, plot=False)

/Users/charlesmartin14/anaconda3/lib/python3.6/site-packages/powerlaw.py:697: RuntimeWarning: divide by zero encountered in true_divide
  (Theoretical_CDF * (1 - Theoretical_CDF))
/Users/charlesmartin14/anaconda3/lib/python3.6/site-packages/powerlaw.py:697: RuntimeWarning: invalid value encountered in true_divide
  (Theoretical_CDF * (1 - Theoretical_CDF))
Assuming nested distributions


alexnet & 17& 2.25 & $(4096\times 9216)$ & 2.29 & 0.0527 & PL \\ 


Assuming nested distributions


  & 20& 1 & $(4096\times 4096)$ & 2.25 & 0.0372 & PL \\ 


Assuming nested distributions
/Users/charlesmartin14/anaconda3/lib/python3.6/site-packages/powerlaw.py:1602: RuntimeWarning: invalid value encountered in true_divide
  CDF = CDF/norm
/Users/charlesmartin14/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)
'nan' in fit cumulative distribution values.

  & 22& 4.1 & $(1000\times 4096)$ & 3.02 & 0.0186 & PL \\ 
\hline



Likely underflow or overflow error: the optimal fit for this distribution gives values that are so extreme that we lack the numerical precision to calculate them.
/Users/charlesmartin14/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


### Generate data for Table

In [6]:
for model in [models.alexnet, 
              models.densenet121, models.densenet161, models.densenet169, models.densenet201, 
              models.inception_v3,
              models.resnet101, models.resnet152, models.resnet18, models.resnet34, models.resnet50, 
              models.squeezenet1_0, models.squeezenet1_1,
              models.vgg11, models.vgg11_bn,
              models.vgg16, models.vgg16_bn,
              models.vgg19, models.vgg19_bn ]:
    #print(model)
    analyze_model(model, plot=False)

/Users/charlesmartin14/anaconda3/lib/python3.6/site-packages/powerlaw.py:697: RuntimeWarning: divide by zero encountered in true_divide
  (Theoretical_CDF * (1 - Theoretical_CDF))
/Users/charlesmartin14/anaconda3/lib/python3.6/site-packages/powerlaw.py:697: RuntimeWarning: invalid value encountered in true_divide
  (Theoretical_CDF * (1 - Theoretical_CDF))
Assuming nested distributions


alexnet & 17& 2.25 & $(4096\times 9216)$ & 2.29 & 0.0527 & PL \\ 


Assuming nested distributions


  & 20& 1 & $(4096\times 4096)$ & 2.25 & 0.0372 & PL \\ 


Assuming nested distributions
/Users/charlesmartin14/anaconda3/lib/python3.6/site-packages/powerlaw.py:1602: RuntimeWarning: invalid value encountered in true_divide
  CDF = CDF/norm
/Users/charlesmartin14/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)
'nan' in fit cumulative distribution values.
Likely underflow or overflow error: the optimal fit for this distribution gives values that are so extreme that we lack the numerical precision to calculate them.
/Users/charlesmartin14/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


...

vgg19 bn & 56& 6.12 & $(4096\times 25088)$ & 2.04 & 0.0295 & TPL \\ 


Assuming nested distributions


  & 59& 1 & $(4096\times 4096)$ & 1.98 & 0.0373 & TPL \\ 


Assuming nested distributions


  & 62& 4.1 & $(1000\times 4096)$ & 3.03 & 0.035 & PL \\ 
\hline
